In [0]:
user_tab_spark = spark.sql("""
  SELECT
    ds.user_id AS user_id,
    --
    COUNT(DISTINCT ds.session_id) AS nr_sessions,
    
    MAX(CASE WHEN ds.status = 'booked_and_travelled' THEN ds.session_start ELSE NULL END) AS date_last_booking_travelled,


    SUM(ds.page_clicks) AS page_clicks_tot,
    AVG(ds.page_clicks) AS page_clicks_avg,

    SUM(CASE 
      WHEN ds.flight_booked = TRUE AND ds.return_flight_booked = TRUE AND ds.status = 'booked_and_travelled' THEN 2 
      WHEN ds.flight_booked = TRUE AND ds.return_flight_booked = FALSE AND ds.status = 'booked_and_travelled' THEN 1
      ELSE 0 END
    ) AS flights_taken, 
    /*the original one wasn't considering return flights and ignoring that cancellation=False doesn't mean that the trip took actual place*/
    
    SUM(CASE 
      WHEN ds.hotel_booked = TRUE AND ds.status = 'booked_and_travelled' THEN 1 
      ELSE 0 END
    ) AS hotels_stayed,
    
    SUM(CASE WHEN ds.status = 'booked_and_travelled' THEN 1 ELSE 0 END) AS nr_booked_and_travelled,
    SUM(CASE WHEN ds.status = 'booked_but_cancelled' THEN 1 ELSE 0 END) AS nr_booked_but_cancelled,
    SUM(CASE WHEN ds.status = 'cancellation_session' THEN 1 ELSE 0 END) AS nr_cancellation_session,
    SUM(CASE WHEN ds.status = 'no_booking' THEN 1 ELSE 0 END) AS nr_no_booking, 




    AVG(CASE WHEN ds.flight_discount = TRUE AND ds.status = 'booked_and_travelled' THEN ds.flight_discount_amount ELSE NULL END) AS flight_taken_discount_avg,
    SUM(CASE WHEN ds.flight_discount = TRUE AND ds.status = 'booked_and_travelled' THEN 1 ELSE 0 END) AS flight_taken_discount_nr, /*this wasn't here*/
        
    AVG(CASE WHEN ds.hotel_discount = TRUE AND ds.status = 'booked_and_travelled' THEN ds.hotel_discount_amount ELSE NULL END) AS hotel_stayed_discount_avg,
    SUM(CASE WHEN ds.hotel_discount = TRUE AND ds.status = 'booked_and_travelled' THEN 1 ELSE 0 END) AS hotel_stayed_discount_nr,/*this wasn't here*/

    SUM(CASE WHEN ds.flight_discount = TRUE AND ds.hotel_discount = TRUE AND ds.status = 'booked_and_travelled' THEN 1 ELSE 0 END) AS hotel_and_flight_discount_nr,/*this wasn't here*/
    
    AVG(CASE WHEN ds.flight_booked = TRUE AND ds.status = 'booked_and_travelled' THEN ds.base_fare_usd ELSE NULL END) AS flight_taken_price_avg, /*only and all those trips with both departure and return flights have flight price*/
    
    AVG(CASE WHEN ds.hotel_booked = TRUE AND ds.status = 'booked_and_travelled' THEN ds.hotel_price_per_room_night_usd ELSE NULL END) AS hotel_stayed_room_price_per_night_avg,



    AVG(CASE WHEN ds.hotel_booked = TRUE AND ds.status = 'booked_and_travelled' THEN ds.nights_calc ELSE NULL END) AS hotel_stayed_nights_avg,

    COUNT(DISTINCT CASE WHEN ds.flight_booked = TRUE AND ds.status = 'booked_and_travelled' THEN ds.trip_airline ELSE NULL END) AS nr_airlines_taken,


    /*distance*/
    SUM(CASE WHEN ds.flight_booked = TRUE AND ds.status = 'booked_and_travelled' THEN ds.distance_km ELSE NULL END) AS distance_km_tot,
    AVG(CASE WHEN ds.flight_booked = TRUE AND ds.status = 'booked_and_travelled' THEN ds.distance_km ELSE NULL END) AS distance_km_avg,

    /*duration*/
    SUM(CASE WHEN ds.duration IS NOT NULL AND ds.status = 'booked_and_travelled' THEN ds.duration ELSE NULL END) AS duration_tot,
    AVG(CASE WHEN ds.duration IS NOT NULL AND ds.status = 'booked_and_travelled' THEN ds.duration ELSE NULL END) AS duration_avg,


    MAX(ds.birthdate) AS birthdate,
    MAX(ds.gender) AS gender,
    BOOL_OR(ds.married) AS married,
    BOOL_OR(ds.has_children) AS has_children,
    MAX(ds.home_country) AS home_country,
    MAX(ds.home_city) AS home_city,
    MAX(ds.home_airport) AS home_airport,
    MAX(ds.home_airport_lat) AS home_airport_lat,
    MAX(ds.home_airport_lon) AS home_airport_lon,
    MAX(ds.sign_up_date) AS sign_up_date,
    
    MAX(ds.age_in_2023) AS age_in_2023,
    MAX(ds.age_bin) AS age_bin,
    
    SUM(CASE WHEN ds.status = 'booked_and_travelled' THEN ds.session_cost ELSE NULL END) AS user_spend_tot,
    AVG(CASE WHEN ds.status = 'booked_and_travelled' THEN ds.session_cost ELSE NULL END) AS user_spend_avg,

    AVG(CASE WHEN ds.booking_window_days IS NOT NULL THEN ds.booking_window_days ELSE NULL END) AS booking_window_days_avg,
    MIN(CASE WHEN ds.booking_window_days IS NOT NULL THEN ds.booking_window_days ELSE NULL END) AS booking_window_days_min,
    MAX(CASE WHEN ds.booking_window_days IS NOT NULL THEN ds.booking_window_days ELSE NULL END) AS booking_window_days_max, 
    /* no need to have ds.status = 'booked_and_travelled' because the "booking_window_days" was created only for "booked and travelled" sessions */

    SUM(CASE WHEN ds.status = 'booked_and_travelled' AND ds.family_session = TRUE THEN 1 ELSE 0 END) AS family_travelled_trip_nr,
    SUM(CASE WHEN ds.status = 'booked_and_travelled' AND ds.family_session = FALSE THEN 1 ELSE 0 END) AS non_family_travelled_trip_nr


    FROM df_sessions_tab ds /*TABLE SAVED AS "sessions_preprocessed.csv"*/ 
    GROUP BY ds.user_id
""")

user_tab_pd = user_tab_spark.toPandas() 


In [0]:
%sql
/* to drop the delta lake table, before recreating it*/
DROP TABLE workspace.default.usertab_from_sessiontab_spark;

In [0]:
user_tab_spark.write.mode("overwrite").saveAsTable("usertab_from_sessiontab_spark") #TABLE SAVED AS "user_RawData.csv"